In [141]:
import os, sys
sys.path.append("../models/")
sys.path.append("../utils/")
sys.path.append("../handle_data/")
import tensorflow as tf
import tensorflow.keras as keras
# all the layers used for U-net
from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate, Conv2D,
                                     Conv2DTranspose, Input, MaxPool2D, Dense, Flatten, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
from unet_model import build_unet, conv_block
from wgan_model import *
from handle_data_unet import *
from handle_data_class import  *
#from plotting import *
from other_utils import provide_default
from typing import List, Tuple, Union
# import climetlab as cml 
import datetime as dt
import numpy as np
import xarray as xr
import json as js
import gc

from collections import OrderedDict

In [ ]:
!pip install climetlab

In [137]:
print(tf.__version__)

2.6.0


# Load the test dataset

In [207]:
datadir = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap5_michael/preprocessed_era5_crea6/netcdf_data/all_files/"
ds_test = xr.open_dataset(os.path.join(datadir, "preproc_era5_crea6_test.nc"))

# Load the trained models

In [225]:
savedir = "/p/project/deepacf/maelstrom/gong1/downscaling_ap5/trained_models/2009_2016_train"
model_name = "2009_2016_train_generator/"
f_dir = os.path.join( savedir, model_name)
new_model = tf.keras.models.load_model(f_dir,compile=False)

In [227]:
# Take small samples for testing
ds_test = ds_test.sel(time=slice("2018-01-01", "2018-01-30"))

In [228]:
groud_truth = ds_test["t_2m_tar"]

In [229]:
#Get the statsitic information from saved json file
js_norm = os.path.join(savedir, "norm_dict.json")
norm_dims = ["time", "lat", "lon"]

with open(js_norm, "r") as f:
    norm_dict = js.load(f)

    
# preprocess data (i.e. normalizing)
def reshape_ds(ds):
    da = ds.to_array(dim="variables")
    da = da.transpose(..., "variables")
    return da

train_vars = list(ds_test.keys())
mu_train, std_train = np.asarray(norm_dict["mu"]), np.asarray(norm_dict["std"])
#mu_train = xr.DataArray(mu_train, coords={"variables": train_vars}, dims=["variables"])
#std_train = xr.DataArray(std_train, coords={"variables": train_vars}, dims=["variables"])
print("mu_train",mu_train)
print("std_train", std_train)
ds_test = reshape_ds(ds_test)
ds_test = HandleUnetData.z_norm_data(ds_test, norm_method="norm", save_path = savedir)


mu_train {'rotated_pole': 1.0, '2t_in': 281.45906591701794, 'sshf_in': -50942.02725137752, 'slhf_in': -163213.65913866778, 'blh_in': 502.4760395386998, '10u_in': 0.6583249490407379, '10v_in': 0.29964431657095814, 'z_in': 5686.443347884734, 't850_in': 277.5147786749324, 't925_in': 281.41634941763175, 'hsurf_tar': 571.5723795572917, 't_2m_tar': 281.918891182312}
std_train {'rotated_pole': 0.0, '2t_in': 8.417833978084378, 'sshf_in': 192018.66261724447, 'slhf_in': 248998.20731436837, 'blh_in': 469.94672652856195, '10u_in': 2.2966472941410414, '10v_in': 1.9018157740395092, 'z_in': 4484.579126295764, 't850_in': 6.938319414004714, 't925_in': 7.52898602570127, 'hsurf_tar': 497.46767440871554, 't_2m_tar': 8.50941746128903}
Loading file: /p/project/deepacf/maelstrom/gong1/downscaling_ap5/trained_models/2009_2016_train/norm_dict.json
norm_dict mu {'rotated_pole': 1.0, '2t_in': 281.45906591701794, 'sshf_in': -50942.02725137752, 'slhf_in': -163213.65913866778, 'blh_in': 502.4760395386998, '10u_in':

In [230]:
ds_test.values.shape
#Split the inputs and output
da_test_in, da_test_tar = HandleDataClass.split_in_tar(ds_test)

# Use trained model and make prediction 

In [231]:
y_pred =  new_model.predict(da_test_in.squeeze().values,batch_size=32)

# Denormalise the prediction values 

In [232]:
# convert predictions to xarray and denormalize
coords = da_test_tar.isel(variables=0).squeeze().coords
dims = da_test_tar.isel(variables=0).squeeze().dims
y_pred_trans = xr.DataArray(y_pred[0].squeeze(), coords=coords, dims=dims)

In [233]:
y_pred_trans

<xarray.DataArray (time: 719, rlat: 96, rlon: 120)>
array([[[-2.75298834e-01, -2.21534610e-01, -3.66437435e-01, ...,
         -9.16103303e-01, -9.11166430e-01, -9.01715934e-01],
        [-2.56428003e-01, -1.27219141e-01, -1.74919724e-01, ...,
         -1.01615310e+00, -1.05708051e+00, -9.72498298e-01],
        [-3.15107286e-01, -1.61646247e-01, -1.57326579e-01, ...,
         -1.09603274e+00, -1.10244536e+00, -1.03186285e+00],
        ...,
        [-2.00557128e-01, -8.88685808e-02, -1.25892729e-01, ...,
         -7.75368437e-02, -6.70922995e-02, -4.83460352e-02],
        [-1.18309982e-01, -6.33112118e-02, -1.06014684e-01, ...,
         -7.59150907e-02, -7.33302310e-02, -6.03354350e-02],
        [ 3.83128002e-02, -9.26874802e-02, -5.90125024e-02, ...,
         -5.39802015e-02, -6.98599890e-02, -3.16873416e-02]],

       [[-3.29975903e-01, -2.73650765e-01, -4.20412064e-01, ...,
         -9.15146708e-01, -9.04359281e-01, -8.97438765e-01],
        [-2.91454673e-01, -1.65480554e-01, -2.13630795e-01, ...,
         -1.00430739e+00, -1.04779363e+00, -9.61987257e-01],
        [-3.49524498e-01, -2.09486485e-01, -2.00263619e-01, ...,
         -1.08708489e+00, -1.09350979e+00, -1.02016091e+00],
...
         -7.50662982e-01, -7.81412423e-01, -8.27274680e-01],
        [-3.25501949e-01, -3.44802976e-01, -3.64331722e-01, ...,
         -7.50737309e-01, -7.36093879e-01, -7.79382706e-01],
        [-2.22402602e-01, -3.77792150e-01, -3.30319375e-01, ...,
         -7.38527477e-01, -7.83598900e-01, -6.24108791e-01]],

       [[-4.98311102e-01, -4.69335318e-01, -5.40547252e-01, ...,
         -5.89527011e-01, -5.55265427e-01, -6.46909118e-01],
        [-4.29883480e-01, -3.21082890e-01, -3.54578853e-01, ...,
         -6.14951730e-01, -6.86348379e-01, -6.48161173e-01],
        [-4.95084643e-01, -3.74651551e-01, -2.82198548e-01, ...,
         -6.33555412e-01, -6.97407246e-01, -6.84098423e-01],
        ...,
        [-2.84280479e-01, -2.21700057e-01, -2.51288593e-01, ...,
         -7.87739873e-01, -8.19902182e-01, -8.69205832e-01],
        [-2.79069066e-01, -2.87860096e-01, -3.02761614e-01, ...,
         -7.90002823e-01, -7.75172114e-01, -8.15972924e-01],
        [-1.72880679e-01, -3.29281569e-01, -2.79383540e-01, ...,
         -7.84282982e-01, -8.24930727e-01, -6.54460669e-01]]],
      dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01T01:00:00 ... 2018-01-30T23:00:00
  * rlon       (rlon) float64 -8.273 -8.218 -8.163 ... -1.838 -1.783 -1.728
  * rlat       (rlat) float64 -3.933 -3.878 -3.823 -3.768 ... 1.182 1.237 1.292
    variables  <U12 't_2m_tar'

In [235]:
y_pred_trans = y_pred_trans.squeeze()*norm_dict["std"]['t_2m_tar'] +norm_dict["mu"]['t_2m_tar']
y_pred_trans = xr.DataArray(y_pred_trans, coords=coords, dims=dims)

In [236]:
groud_truth.values

In [237]:
y_pred_trans

<xarray.DataArray (time: 719, rlat: 96, rlon: 120)>
array([[[279.57626, 280.03375, 278.80072, ..., 274.12338, 274.16537,
         274.24582],
        [279.73682, 280.83633, 280.43042, ..., 273.272  , 272.92374,
         273.6435 ],
        [279.23752, 280.54337, 280.58014, ..., 272.5923 , 272.53772,
         273.13834],
        ...,
        [280.21225, 281.16266, 280.8476 , ..., 281.2591 , 281.34796,
         281.50748],
        [280.91214, 281.38016, 281.01675, ..., 281.2729 , 281.2949 ,
         281.40546],
        [282.2449 , 281.13016, 281.41672, ..., 281.45953, 281.3244 ,
         281.64923]],

       [[279.111  , 279.59027, 278.34143, ..., 274.13153, 274.22333,
         274.2822 ],
        [279.43878, 280.51074, 280.101  , ..., 273.3728 , 273.00278,
         273.73294],
        [278.94464, 280.1363 , 280.21475, ..., 272.66843, 272.61374,
         273.2379 ],
...
        [279.10284, 279.55457, 279.34973, ..., 275.5312 , 275.26953,
         274.87927],
        [279.14905, 278.9848 , 278.81863, ..., 275.53055, 275.65515,
         275.2868 ],
        [280.02637, 278.7041 , 279.10806, ..., 275.63446, 275.25092,
         276.6081 ]],

       [[277.67856, 277.9251 , 277.31915, ..., 276.90234, 277.1939 ,
         276.41406],
        [278.26083, 279.18665, 278.9016 , ..., 276.686  , 276.07846,
         276.4034 ],
        [277.706  , 278.7308 , 279.51755, ..., 276.5277 , 275.98434,
         276.0976 ],
        ...,
        [279.49982, 280.03235, 279.78058, ..., 275.21567, 274.942  ,
         274.52246],
        [279.54416, 279.46936, 279.34256, ..., 275.1964 , 275.32263,
         274.97543],
        [280.44778, 279.11688, 279.5415 , ..., 275.2451 , 274.8992 ,
         276.3498 ]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 2018-01-01T01:00:00 ... 2018-01-30T23:00:00
  * rlon       (rlon) float64 -8.273 -8.218 -8.163 ... -1.838 -1.783 -1.728
  * rlat       (rlat) float64 -3.933 -3.878 -3.823 -3.768 ... 1.182 1.237 1.292
    variables  <U12 't_2m_tar'

In [238]:
mse_mean, mse_std = np.zeros(24), np.zeros(24)

for i, hh in enumerate(np.arange(0, 24)):
    #mse_all = ((y_pred_trans.sel(time=dt.time(hh)) - groud_truth.sel(time=dt.time(hh)))**2).mean(dim=["lat", "lon"])
    mse_all = ((y_pred_trans.sel(time=dt.time(hh)) - groud_truth.sel(time=dt.time(hh)))**2)
    mse_mean[i], mse_std[i] = mse_all.mean().values, mse_all.std().values

In [242]:
 mse_mean

<xarray.DataArray (daytime: 24)>
array([0.98208594, 1.01417267, 1.03898025, 1.06031191, 1.08137274,
       1.09343898, 1.06370151, 1.05588412, 1.01716602, 1.04859614,
       1.19139981, 1.41016686, 1.45807362, 1.3850342 , 1.27499545,
       1.1108005 , 0.9343524 , 0.89572787, 0.88000661, 0.89816785,
       0.93669784, 0.98314506, 0.99995327, 1.06978059])
Coordinates:
  * daytime  (daytime) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23

In [243]:
mse_std

<xarray.DataArray (daytime: 24)>
array([1.78109407, 1.85888278, 1.89970231, 1.9450779 , 1.94259107,
       1.92870498, 1.88588929, 1.87938833, 1.83949709, 1.98989391,
       2.35744524, 2.79234958, 2.95166492, 2.91424227, 2.61826587,
       2.17255831, 1.80107462, 1.70245659, 1.62200737, 1.59896576,
       1.67204642, 1.76832139, 1.81939971, 1.91112852])
Coordinates:
  * daytime  (daytime) int64 0 1 2 3 4 5 6 7 8 9 ... 15 16 17 18 19 20 21 22 23

# Visualize the results

In [244]:
from matplotlib import pyplot as plt

mse_mean = xr.DataArray(mse_mean, coords = {"daytime": np.arange(0,24)}, dims=["daytime"]) 
mse_std = xr.DataArray(mse_std, coords = {"daytime": np.arange(0,24)}, dims=["daytime"])

mse_mean_v, mse_std_v = mse_mean.values, mse_std.values
fig, (ax) = plt.subplots(1,1)
ax.plot(mse_mean["daytime"].values, mse_mean_v, 'k-', label="ERA5 DeepHRES")
ax.fill_between(mse_mean["daytime"].values, mse_mean_v-mse_std_v, mse_mean_v+mse_std_v, facecolor="blue", alpha=0.2)
ax.set_ylim(0.,4.)
# label axis
ax.set_xlabel("daytime [UTC]", fontsize=16)
ax.set_ylabel("MSE T2m [K$^2$]", fontsize=16)
ax.tick_params(axis="both", which="both", direction="out", labelsize=14)

## add MSE from previous non-augmented dataset
#ax.errorbar(12, 0.394, yerr=0.094, fmt='x', capsize=5., ecolor="black", mfc="red",
#            mec="red", ms=10, mew=2., label = "Unet small")
# Configure legend
# get handles
handles, labels = ax.get_legend_handles_labels()
# remove the errorbars
#handles[1] = handles[1][0]

ax.legend(handles, labels, loc='upper right', numpoints=1)
# save plot to file
fig.savefig("downscaling_wgan_t2m_mse.png")

In [ ]:
# choose a time index
tind = 380

y_pred_eval = y_pred_trans#.sel(time=dt.time(12))

# plot the full 2m temperature
plt_fname_exp = "./plot_temp_pred_real"
create_plots(y_pred_eval.isel(time=tind), ds_test["t2m_tar"].isel(time=tind), plt_fname_exp,
             opt_plot={"title1": "downscaled T2m", "title2": "target T2m", "levels": np.arange(-3, 27., 1.)})

plt_fname_diff = "./plot_temp_diff"
diff_in_tar = ds_test["2t_in"].isel(time=tind)-ds_test["t2m_tar"].isel(time=tind) + 273.15
diff_down_tar = y_pred_eval.isel(time=tind)-ds_test["t2m_tar"].isel(time=tind) + 273.15
create_plots(diff_in_tar, diff_down_tar, plt_fname_diff,
             opt_plot={"title1": "diff. input-target", "title2": "diff. downscaled-target",
                       "levels": np.arange(-3., 3.1, .2)})